In [1]:
# Install packages

#import sys
#!{sys.executable} -m pip install selenium
#!{sys.executable} -m pip install webdriver_manager

In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re
import numpy as np

In [3]:
service=Service(ChromeDriverManager().install())

In [4]:
def get_jobs(keyword, location, num_jobs, verbose, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    jobs = []
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder by changing service=(addres)
    #Chrome driver is being installed everytime the kernel is restart and saved as variable, 
    #not needing to provide location path
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1120, 1000)
    
    # Glassdoor website URL
    # This program only works for US and Canada due to glassdoor requiring location id for proper search
    if location.lower() == 'canada':
        # Below is the url location tags for jobs in Canada
        loc = "&locT=N&locId=3&locName=Canada"
    elif location.lower() == 'us' or location.lower() == 'usa':
        # Below is the url location tags for jobs in US
        loc = "&locT=N&cId=1&locName=United%20States"
    else:
        print("Invalid Location: Canada or US only")
        return pd.DataFrame(jobs)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+loc+"&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    

    #If true, should be still looking for new jobs.
    while len(jobs) < num_jobs:  

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.CLASS_NAME,"selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(1)

        try:
            driver.find_element(By.CSS_SELECTOR,'[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass

        
        #Gather all the job node elements from the left column of Glassdoor
        #These are the buttons we're going to click.
        job_buttons = driver.find_elements(By.XPATH, "//article[@id='MainCol']//ul/li[starts-with(@class, 'react-job-listing')]")  
        
        for job_button in job_buttons:  
            
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            #Click on each job node element
            webdriver.ActionChains(driver).move_to_element(job_button).click(job_button).perform()
            
            time.sleep(1)
            collected_successfully = False
            
            #Gather the basic information from the loaded job posting main page
            while not collected_successfully:
                try:
                    try:
                        company_name = driver.find_element(By.XPATH, './/div[@data-test="employerName"]').text
                    except NoSuchElementException:
                        company_name = -1
                    try:
                        division = driver.find_element(By.XPATH, './/div[@class="division"]').text
                    except NoSuchElementException:
                        division = -1
                    try:
                        location = driver.find_element(By.XPATH, './/div[@data-test="location"]').text
                    except NoSuchElementException:
                        location = -1
                    try:
                        job_title = driver.find_element(By.XPATH, './/div[@data-test="jobTitle"]').text
                    except NoSuchElementException:
                        job_title = -1
                    try:
                        driver.find_element(By.XPATH, './/div[@id="JobDescriptionContainer"]//div[text()="Show More"]').click()
                        tim.sleep(0.5)
                    except NoSuchElementException:
                        pass
                    try:
                        job_description = driver.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text
                    except NoSuchElementException:
                        job_description = -1
                        
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                #Glassdoor salary estimate
                salary_estimate = job_button.find_element(By.XPATH, './/span[@data-test="detailSalary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                #Glassdoor employer review rating score
                rating = driver.find_element(By.XPATH, './/div[@id="employerStats"]/div[1]/div').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Division: {}".format(division))
                print("Location: {}".format(location))


            #Gather additional information about the company from Glassdoor company overview section
            try:
                driver.find_element(By.XPATH, './/div[@id="CompanyContainer"]')

                try:
                    size = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]//div[span/text()="Size"]/span[2]').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]//div[span/text()="Founded"]/span[2]').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]//div[span/text()="Type"]/span[2]').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]//div[span/text()="Industry"]/span[2]').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]//div[span/text()="Sector"]/span[2]').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]//div[span/text()="Revenue"]/span[2]').text
                except NoSuchElementException:
                    revenue = -1


            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : float(rating),
            "Company Name" : company_name,
            "Division" : division,             
            "Location" : location,
            "Size" : size,
            "Founded" : int(founded),
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            #add job to jobs
                
        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH, "//article[@id='MainCol']//button[starts-with(@class, 'nextButton')]").click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
   
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [5]:
# Can change the salary part to get the entire range estimate rather than average
# Company descriptors do not necessarily include the same factors listed (line 113-145)
# As the section uses div indexing, the values may not match with correct column
# Can instead perform an evaluation based on the sub-heading

# https://stackoverflow.com/questions/3655549/xpath-containstext-some-string-doesnt-work-when-used-with-node-with-more
# //*[contains(text(),'ABC')]

# try:
#    driver.findElement.textcontains('industry')
#    try:
#        industry = driver.findElement(industry).text
#    except NoSuchElementException:
#        industry = None
# except NoSuchElementException:
#     industry = None

In [6]:
# Change the location keyword to either "canada" or "us"

#df = get_jobs('data scientist','canada',10, False, 15)
df = get_jobs('data scientist','canada',1000, False, 15)

df.to_csv('glassdoor_data_scientist_canada_221117_1000.csv', index = False)

 x out failed
Progress: 0/1000
Progress: 1/1000
Progress: 2/1000
Progress: 3/1000
Progress: 4/1000
Progress: 5/1000
Progress: 6/1000
Progress: 7/1000
Progress: 8/1000
Progress: 9/1000
Progress: 10/1000
Progress: 11/1000
Progress: 12/1000
Progress: 13/1000
Progress: 14/1000
Progress: 15/1000
Progress: 16/1000
Progress: 17/1000
Progress: 18/1000
Progress: 19/1000
Progress: 20/1000
Progress: 21/1000
Progress: 22/1000
Progress: 23/1000
Progress: 24/1000
Progress: 25/1000
Progress: 26/1000
Progress: 27/1000
Progress: 28/1000
Progress: 29/1000
 x out failed
Progress: 30/1000
Progress: 31/1000
Progress: 32/1000
Progress: 33/1000
Progress: 34/1000
Progress: 35/1000
Progress: 36/1000
Progress: 37/1000
Progress: 38/1000
Progress: 39/1000
Progress: 40/1000
Progress: 41/1000
Progress: 42/1000
Progress: 43/1000
Progress: 44/1000
Progress: 45/1000
Progress: 46/1000
Progress: 47/1000
Progress: 48/1000
Progress: 49/1000
Progress: 50/1000
Progress: 51/1000
Progress: 52/1000
Progress: 53/1000
Progress: 

Progress: 426/1000
Progress: 427/1000
Progress: 428/1000
Progress: 429/1000
Progress: 430/1000
Progress: 431/1000
Progress: 432/1000
Progress: 433/1000
Progress: 434/1000
Progress: 435/1000
Progress: 436/1000
Progress: 437/1000
Progress: 438/1000
Progress: 439/1000
Progress: 440/1000
Progress: 441/1000
Progress: 442/1000
Progress: 443/1000
Progress: 444/1000
Progress: 445/1000
Progress: 446/1000
Progress: 447/1000
Progress: 448/1000
Progress: 449/1000
 x out failed
Progress: 450/1000
Progress: 451/1000
Progress: 452/1000
Progress: 453/1000
Progress: 454/1000
Progress: 455/1000
Progress: 456/1000
Progress: 457/1000
Progress: 458/1000
Progress: 459/1000
Progress: 460/1000
Progress: 461/1000
Progress: 462/1000
Progress: 463/1000
Progress: 464/1000
Progress: 465/1000
Progress: 466/1000
Progress: 467/1000
Progress: 468/1000
Progress: 469/1000
Progress: 470/1000
Progress: 471/1000
Progress: 472/1000
Progress: 473/1000
Progress: 474/1000
Progress: 475/1000
Progress: 476/1000
Progress: 477/100

Progress: 847/1000
Progress: 848/1000
Progress: 849/1000
Progress: 850/1000
Progress: 851/1000
Progress: 852/1000
Progress: 853/1000
Progress: 854/1000
Progress: 855/1000
Progress: 856/1000
Progress: 857/1000
Progress: 858/1000
Progress: 859/1000
Progress: 860/1000
Progress: 861/1000
Progress: 862/1000
Progress: 863/1000
Progress: 864/1000
Progress: 865/1000
Progress: 866/1000
Progress: 867/1000
Progress: 868/1000
Progress: 869/1000
 x out failed
Progress: 870/1000
Progress: 871/1000
Progress: 872/1000
Progress: 873/1000
Progress: 874/1000
Progress: 875/1000
Progress: 876/1000
Progress: 877/1000
Progress: 878/1000
Progress: 879/1000
Progress: 880/1000
Progress: 881/1000
Progress: 882/1000
Progress: 883/1000
Progress: 884/1000
Progress: 885/1000
Progress: 886/1000
Progress: 887/1000
Progress: 888/1000
Progress: 889/1000
Progress: 890/1000
Progress: 891/1000
Progress: 892/1000
Progress: 893/1000
Progress: 894/1000
Progress: 895/1000
Progress: 896/1000
Progress: 897/1000
Progress: 898/100

In [7]:
df.head(10)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Division,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,CA$55K - CA$65K (Glassdoor est.),Charger Logistics is a world class asset-based...,3.5,Charger Logistics Inc\n3.5,-1,Brampton,1001 to 5000 Employees,2005,Company - Private,Taxi & Car Services,Transportation & Logistics,$5 to $25 million (USD)
1,Junior Data Scientist,CA$74K - CA$90K (Glassdoor est.),We are looking for our next Junior Data Scient...,3.8,M&M Food Market\n3.8,-1,Mississauga,1001 to 5000 Employees,-1,Company - Private,Animal Production,Agriculture,$25 to $100 million (USD)
2,Data Scientist,CA$55K - CA$122K (Glassdoor est.),Join our Winning Team as a Data Scientist\nWhe...,4.5,Carfax\n4.5,-1,London,1001 to 5000 Employees,1984,Company - Public,Internet & Web Services,Information Technology,Unknown / Non-Applicable
3,Applied Data Scientist,CA$55K (Employer est.),Position\nApplied Data Scientist\nReporting to...,-1.0,Airudi,-1,Remote,-1,-1,-1,-1,-1,-1
4,Analytics Data Scientist,CA$65K - CA$100K (Employer est.),BrainFinance is a leading financial technology...,-1.0,Brain Finance,-1,Montreal,Unknown,-1,Company - Private,-1,-1,Unknown / Non-Applicable
5,Data Scientist,CA$85K - CA$120K (Glassdoor est.),Referred applicants should not apply directly ...,3.6,Loblaw Companies Limited\n3.6,-1,Brampton,10000+ Employees,-1,Company - Public,Vehicle Dealers,Retail & Wholesale,$10+ billion (USD)
6,Machine Learning Engineer,CA$74K - CA$97K (Glassdoor est.),Charger Logistics is a world class asset-based...,3.5,Charger Logistics Inc\n3.5,-1,Brampton,1001 to 5000 Employees,2005,Company - Private,Taxi & Car Services,Transportation & Logistics,$5 to $25 million (USD)
7,"Sr. Data Scientist - Up to $110,000 - Toronto",CA$110K (Employer est.),Role: Sr. Data Scientist\nStructure: Permanent...,5.0,CorGTA Inc.\n5.0,-1,Toronto,1 to 50 Employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable
8,"SENIOR MANAGER, DATA & ARTIFICIAL INTELLIGENCE...",CA$71K - CA$92K (Glassdoor est.),"Location: Pickering, ON, CA, L1W 3J2\nReq ID: ...",4.2,Ontario Power Generation\n4.2,-1,Pickering,5001 to 10000 Employees,1999,Government,Energy & Utilities,"Energy, Mining & Utilities",$1 to $5 billion (USD)
9,Software Engineer in Algorithms & Optimization...,CA$90K - CA$110K (Employer est.),Role Summary:\nThis is an opportunity in the e...,4.1,RideCo\n4.1,-1,Waterloo,51 to 200 Employees,2013,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable


In [8]:
df.dtypes

Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Division               int64
Location              object
Size                  object
Founded                int64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
dtype: object

In [29]:
df.iloc[227]

Job Title            -1
Salary Estimate      -1
Job Description      -1
Rating               -1
Company Name         -1
Division             -1
Location             -1
Size                 -1
Founded              -1
Type of ownership    -1
Industry             -1
Sector               -1
Revenue              -1
Name: 227, dtype: object

In [21]:
# url2 = "https://www.glassdoor.com/Job/canada-data-scientist-jobs-SRCH_IL.0,6_IN3_KO7,21_IP6.htm?includeNoSalaryJobs=true&pgc=AB4ABYEAlgAAAAAAAAAAAAAAAe5EeBQAuQEDARs4CzoGfhJTbiC5ApjWmIvkwH1s1z3uEPNZzSGKaHAxaZM5%2F5DRVe8pQ1%2BsJPUTeE4IuXG8UtUIgJ6mDPs9y%2B%2Bv0vYftzSodDRaF4hgccAHeXCtjYhMzWT%2F4iey7pomOSCVRrWsTBjbZwk1B9PpR%2BTKkE6L30NLRundHM7eEfmOfMSwGn8yBHdRkJPRYHk5bGBwE%2B0aExGZ%2B3TyREdF0kq0eylv1MTOuvKxPKomfoG1hRi%2FXi9IAAA%3D"
# options = webdriver.ChromeOptions()
# driver2 = webdriver.Chrome(service=service, options=options)
# driver2.set_window_size(1120, 1000)
# driver2.get(url2)

In [11]:
# driver2.find_element(By.XPATH, './/div[@id="JobDescriptionContainer"]//div[text()="Show More"]').click()

In [12]:
# print(driver2.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text)